# Set up

### Import Python packages to use in the script

In [20]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
import skan
from imaris_ims_file_reader.ims import ims
from tqdm import tqdm

### Define the functions that will run different steps/tasks on the data

Need to select which GPU device to send data to

In [2]:
cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              573.24
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

In [3]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [29]:
def get_measurements(mask,img,props,voxel):
    filtered_masks = sk.segmentation.clear_border(mask)
    #size_filtered = sk.morphology.remove_small_objects(filtered_masks,min_size=2000)
    df = sk.measure.regionprops_table(filtered_masks,img,properties=props,spacing=voxel)
    df = pd.DataFrame.from_dict(df)
    return filtered_masks, df

In [5]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [27]:
def get_3D_surfacearea_and_sphericity(mask_img,df,voxel):
    surf_area_list = []
    sphericity_list = []
    vol = np.asarray(df['area']).astype(np.float64)
    c = 0
    for i in np.asarray(df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=voxel)
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphericity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphericity_list.append(sphericity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphericities = pd.Series(sphericity_list,name='Sphericity')
    merged_df = pd.concat([df,surface_areas,sphericities], axis=1)
    return merged_df

Below cell contains the formula for calculating sphericity and the source of the code. Not required to run below cell for script to work.

In [ ]:
# def sphericity(mesh_volume, mesh_surface_area):
    
#     '''This definition of sphericity assumes that you are working in continuous space.
#     Parameters:
#     -----------
#     mesh_volume: integer or float value
#     mesh_surface_area: integer or float value
#     Returns:
#     --------
#     psi: a float value with range(0,1) reflecting the compactness of an object
#     Got this script from: https://github.com/BiAPoL/Bio-image_Analysis_with_Python/blob/main/05_feature_extraction/02_sphericity_and_solidity.ipynb
#     '''
#     numerator = (np.pi ** (1/3)) * ((6 * mesh_volume) ** (2/3))
#     denominator = mesh_surface_area
    
#     psi = numerator / denominator
    
#     return psi

In [9]:
def save(save_path, img_name, norm_img, filtered_masks, merged_df):
    masks_path = os.path.join(save_path,'masks')
    norm_path = os.path.join(save_path,'norm_imgs')
    dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    imsave(os.path.join(norm_path,'normalized_'+img_name[:-4]+'.tif'),norm_img)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Load in Data

## Two options for loading in data:
Note that options 1 and 2 expect the images to be in a tiff format. sk.io.imread cannot read in ims files

### Option 1: Specify the locations of all image containing folders:
Then merge into a long list of images and file names

In [ ]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [ ]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [ ]:
all_files = CMO_files+GMO_1ng_files+GMO_2ng_files # need this for getting image names later
all_img = CMO_imgs+GMO_1ng_imgs+GMO_2ng_imgs

### Option 2: Use a recursive search option to find all images in the parent directory:

In [ ]:
all_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/**/*.tif',recursive=True)) #finds all tiff images in the subdirectories of TIF_Files
all_img = list(map(sk.io.imread,all_files))

### Option 3: Read in Imaris files directly

In [13]:
img_array = img[0,0,:,:,:]

In [14]:
img_array.shape

(32, 2048, 2048)

In [14]:
ims_files = sorted(glob('E:/ParadaKusz_Lab/IMS_Files/*.ims'))
imgs = [ims(file,ResolutionLevelLock=0,write=True) for file in ims_files]
all_img = [imgs[i][0,0,:,:,:] for i in range(len(imgs))]

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_1.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_2.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_3.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_4.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_5.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_6.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\control MO_7.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng_1.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng_2.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng_3.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng_4.ims 

Opening writeable file: E:/ParadaKusz_Lab/IMS_Files\gnas MO 1 ng_5.ims 

# Run Pipeline

### Optional use of random integer generator if you want to test on single random images first

In [15]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))

In [16]:
tophat_radius = 10
img = normalize_images(all_img[i],tophat_radius) #change the tophat_radius to increase or decrease the background as needed

In [17]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="Microglia_LargeZ_V2") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(img, channels=channels, z_axis=0,stitch_threshold=0.05, flow_threshold=0.4,cellprob_threshold=0.0)

creating new log file
2025-06-26 12:37:17,662 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-26 12:37:17,664 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-26 12:37:17,666 [INFO] >> Microglia_LargeZ_V2 << model set to be used
2025-06-26 12:37:17,771 [INFO] ** TORCH CUDA version installed and working. **
2025-06-26 12:37:17,773 [INFO] >>>> using GPU (CUDA)
2025-06-26 12:37:17,874 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\Microglia_LargeZ_V2
2025-06-26 12:37:18,034 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-26 12:37:18,035 [INFO] >>>> model diam_labels =  43.143 (mean diameter of training ROIs)
2025-06-26 12:37:18,037 [INFO] multi-stack tiff read in as having 41 planes 1 channels
2025-06-26 12:37:42,375 [INFO] 100%|##########| 41/41 [00:22<00:00,  1.85it/s]
2025-06-26 12:37:44,280 [INFO] network run in 24.07s


100%|██████████| 40/40 [00:02<00:00, 15.31it/s]


2025-06-26 12:38:16,020 [INFO] masks created in 31.73s


In [18]:
#View results of segmentation in Napari
viewer = napari.view_image(all_img[i], name='original img', scale = (4.55,0.3,0.3))
viewer.add_image(img,name='normalized img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

<Image layer 'masks' at 0x2f814985150>

2025-06-26 12:38:58,272 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


### Run pipeline on all images

Set up a few parameters before running

Properties that can be measured with scikit-image can be found here: [Scikit-image Region Properties](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops)

In [19]:
tophat_radius = 10
voxel = [4.55,0.301,0.301]
save_path = 'E:/ParadaKusz_Lab/Outputs/'
props = ('label','area','intensity_mean','intensity_min','intensity_max')

In [21]:
norm_imgs = [normalize_images(img,tophat_radius) for img in tqdm(all_img)]

100%|██████████| 25/25 [04:26<00:00, 10.68s/it]


In [22]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="Microglia_LargeZ_V2") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(norm_imgs, channels=channels, z_axis=0,stitch_threshold=0.05, flow_threshold=0.4,cellprob_threshold=0.0)

creating new log file
2025-06-26 12:47:12,152 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-26 12:47:12,153 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-26 12:47:12,156 [INFO] >> Microglia_LargeZ_V2 << model set to be used
2025-06-26 12:47:12,158 [INFO] ** TORCH CUDA version installed and working. **
2025-06-26 12:47:12,159 [INFO] >>>> using GPU (CUDA)
2025-06-26 12:47:12,254 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\Microglia_LargeZ_V2
2025-06-26 12:47:12,386 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-26 12:47:12,387 [INFO] >>>> model diam_labels =  43.143 (mean diameter of training ROIs)
2025-06-26 12:47:12,390 [INFO] 0%|          | 0/25 [00:00<?, ?it/s]
2025-06-26 12:47:12,390 [INFO] multi-stack tiff read in as having 11 planes 1 channels
2025-06-26 12:47:12,990 [INFO] 
2025-06-26 12:47:12,991 [INFO] 0

100%|██████████| 10/10 [00:00<00:00, 60.77it/s]


2025-06-26 12:47:23,481 [INFO] masks created in 4.14s
2025-06-26 12:47:27,862 [INFO] multi-stack tiff read in as having 32 planes 1 channels
2025-06-26 12:47:29,600 [INFO] 
2025-06-26 12:47:29,602 [INFO] 0%|          | 0/32 [00:00<?, ?it/s]
2025-06-26 12:47:29,603 [INFO] 
2025-06-26 12:47:46,629 [INFO] 100%|##########| 32/32 [00:17<00:00,  1.88it/s]
2025-06-26 12:47:48,134 [INFO] network run in 18.53s
2025-06-26 12:47:48,136 [INFO] 
2025-06-26 12:47:48,137 [INFO] 0%|          | 0/32 [00:00<?, ?it/s]
2025-06-26 12:47:48,139 [INFO] 
2025-06-26 12:48:07,934 [INFO] 100%|##########| 32/32 [00:19<00:00,  1.62it/s]
2025-06-26 12:48:07,934 [INFO] stitching 32 planes using stitch_threshold=0.050 to make 3D masks


100%|██████████| 31/31 [00:01<00:00, 26.70it/s]


2025-06-26 12:48:11,066 [INFO] masks created in 22.93s
2025-06-26 12:48:22,835 [INFO] 8%|8         | 2/25 [01:10<13:30, 35.22s/it]
2025-06-26 12:48:22,836 [INFO] multi-stack tiff read in as having 35 planes 1 channels
2025-06-26 12:48:24,664 [INFO] 
2025-06-26 12:48:24,666 [INFO] 0%|          | 0/35 [00:00<?, ?it/s]
2025-06-26 12:48:24,668 [INFO] 
2025-06-26 12:48:43,174 [INFO] 100%|##########| 35/35 [00:18<00:00,  1.89it/s]
2025-06-26 12:48:44,891 [INFO] network run in 20.23s
2025-06-26 12:48:44,892 [INFO] 
2025-06-26 12:48:44,894 [INFO] 0%|          | 0/35 [00:00<?, ?it/s]
2025-06-26 12:48:44,895 [INFO] 
2025-06-26 12:49:06,748 [INFO] 100%|##########| 35/35 [00:21<00:00,  1.60it/s]
2025-06-26 12:49:06,750 [INFO] stitching 35 planes using stitch_threshold=0.050 to make 3D masks


...

2025-06-26 13:12:01,848 [INFO] masks created in 22.61s
2025-06-26 13:12:18,558 [INFO] 92%|#########2| 23/25 [25:06<02:20, 70.34s/it]
2025-06-26 13:12:18,559 [INFO] multi-stack tiff read in as having 66 planes 1 channels
2025-06-26 13:12:22,259 [INFO] 
2025-06-26 13:12:22,261 [INFO] 0%|          | 0/66 [00:00<?, ?it/s]
2025-06-26 13:12:22,264 [INFO] 
2025-06-26 13:12:52,278 [INFO] 
2025-06-26 13:12:52,280 [INFO] 85%|########4 | 56/66 [00:30<00:05,  1.87it/s]
2025-06-26 13:12:52,282 [INFO] 
2025-06-26 13:12:57,615 [INFO] 100%|##########| 66/66 [00:35<00:00,  1.87it/s]
2025-06-26 13:13:00,806 [INFO] network run in 38.55s
2025-06-26 13:13:00,808 [INFO] 
2025-06-26 13:13:00,810 [INFO] 0%|          | 0/66 [00:00<?, ?it/s]
2025-06-26 13:13:00,812 [INFO] 
2025-06-26 13:13:31,255 [INFO] 
2025-06-26 13:13:31,258 [INFO] 73%|#######2  | 48/66 [00:30<00:11,  1.58it/s]
2025-06-26 13:13:31,260 [INFO] 
2025-06-26 13:13:35,895 [WARNING] no seeds found in get_masks_torch - no masks found.
2025-06-26 13:

100%|██████████| 65/65 [00:02<00:00, 28.45it/s]


2025-06-26 13:13:44,377 [INFO] masks created in 43.57s
2025-06-26 13:14:09,316 [INFO] 96%|#########6| 24/25 [26:56<01:22, 82.47s/it]
2025-06-26 13:14:09,319 [INFO] multi-stack tiff read in as having 52 planes 1 channels
2025-06-26 13:14:12,273 [INFO] 
2025-06-26 13:14:12,275 [INFO] 0%|          | 0/52 [00:00<?, ?it/s]
2025-06-26 13:14:12,276 [INFO] 
2025-06-26 13:14:39,992 [INFO] 100%|##########| 52/52 [00:27<00:00,  1.88it/s]
2025-06-26 13:14:42,392 [INFO] network run in 30.12s
2025-06-26 13:14:42,394 [INFO] 
2025-06-26 13:14:42,396 [INFO] 0%|          | 0/52 [00:00<?, ?it/s]
2025-06-26 13:14:42,398 [INFO] 
2025-06-26 13:14:42,416 [INFO] No cell pixels found.
2025-06-26 13:14:42,464 [INFO] No cell pixels found.
2025-06-26 13:15:06,075 [WARNING] no seeds found in get_masks_torch - no masks found.
2025-06-26 13:15:06,883 [WARNING] no seeds found in get_masks_torch - no masks found.
2025-06-26 13:15:06,925 [INFO] No cell pixels found.
2025-06-26 13:15:06,964 [INFO] No cell pixels found.


100%|██████████| 51/51 [00:01<00:00, 27.88it/s]


2025-06-26 13:15:12,329 [INFO] masks created in 29.94s
2025-06-26 13:15:31,546 [INFO] 100%|##########| 25/25 [28:19<00:00, 67.97s/it]


In [30]:
for i in tqdm(range(len(norm_imgs))):
    img_name = os.path.basename(ims_files[i])
    mask_img, df = get_measurements(masks[i],all_img[i],props,voxel=voxel)
    merged_df = get_3D_surfacearea_and_sphericity(mask_img,df,voxel=voxel)
    save(save_path, img_name, norm_imgs[i], mask_img, merged_df)

100%|██████████| 25/25 [1:29:35<00:00, 215.04s/it]
